In [1]:
from fredapi import Fred
import yfinance as yf
import pandas as pd
from datetime import datetime

In [2]:
FRED_API_KEY = "adf5faa6d759eb1f4b335cef60ade574"
START_DATE = "2006-01-01"
END_DATE = datetime.today().strftime("%Y-%m-%d")

fred = Fred(api_key=FRED_API_KEY)

# -----------------------------------------
# DOWNLOAD FRED MACRO DATA
# -----------------------------------------
dxy = fred.get_series("DTWEXBGS", observation_start=START_DATE)
real_yield = fred.get_series("DFII10", observation_start=START_DATE)
breakeven = fred.get_series("T10YIE", observation_start=START_DATE)
fed_funds = fred.get_series("DFF", observation_start=START_DATE)
oil = fred.get_series("DCOILWTICO", observation_start=START_DATE)

cpi = fred.get_series("CPIAUCSL", observation_start=START_DATE)
m2 = fred.get_series("M2SL", observation_start=START_DATE)

# -----------------------------------------
# TRANSFORM MONTHLY SERIES
# -----------------------------------------
cpi = cpi.resample("M").last()
m2 = m2.resample("M").last()

cpi_yoy = cpi.pct_change(12) * 100
m2_yoy = m2.pct_change(12) * 100

# Lag monthly macro to avoid look-ahead bias
cpi_yoy = cpi_yoy.shift(1)
m2_yoy = m2_yoy.shift(1)

# -----------------------------------------
# DOWNLOAD MARKET DATA (YAHOO)
# -----------------------------------------
gold = yf.download("GC=F", start=START_DATE, end=END_DATE)["Close"]
sp500 = yf.download("^GSPC", start=START_DATE, end=END_DATE)["Close"]
vix = yf.download("^VIX", start=START_DATE, end=END_DATE)["Close"]

# -----------------------------------------
# COMBINE DAILY DATA
# -----------------------------------------
daily_data = pd.concat(
    [gold, dxy, real_yield, breakeven, fed_funds, oil, sp500, vix],
    axis=1
)

daily_data.columns = [
    "Gold",
    "DXY",
    "Real_Yield_10Y",
    "Breakeven_10Y",
    "Fed_Funds",
    "WTI_Oil",
    "SP500",
    "VIX"
]

# Forward-fill macro gaps (weekends, holidays)
daily_data = daily_data.ffill()

# -----------------------------------------
# ADD MONTHLY MACRO (CPI & M2)
# -----------------------------------------
monthly_macro = pd.concat([cpi_yoy, m2_yoy], axis=1)
monthly_macro.columns = ["CPI_YoY", "M2_YoY"]

monthly_macro_daily = monthly_macro.resample("D").ffill()

# -----------------------------------------
# FINAL DATASET
# -----------------------------------------
final_data = pd.concat([daily_data, monthly_macro_daily], axis=1)
final_data = final_data.dropna()

#print(final_data.tail())
#print("\nShape:", final_data.shape)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
print(final_data.tail())
print("\nShape:", final_data.shape)

                   Gold       DXY  Real_Yield_10Y  Breakeven_10Y  Fed_Funds  \
2026-01-27  5079.899902  117.4523            1.90           2.34       3.64   
2026-01-28  5301.600098  117.5117            1.90           2.36       3.64   
2026-01-29  5318.399902  117.4396            1.89           2.35       3.64   
2026-01-30  4713.899902  117.8996            1.90           2.36       3.64   
2026-01-31  4713.899902  117.8996            1.90           2.36       3.64   

            WTI_Oil        SP500        VIX   CPI_YoY    M2_YoY  
2026-01-27    62.04  6978.600098  16.350000  2.696444  3.922108  
2026-01-28    62.75  6978.029785  16.350000  2.696444  3.922108  
2026-01-29    64.77  6969.009766  16.879999  2.696444  3.922108  
2026-01-30    64.50  6939.029785  17.440001  2.696444  3.922108  
2026-01-31    64.50  6939.029785  17.440001  2.653304  4.230511  

Shape: (6913, 10)


In [14]:
final_data.head()

,Gold,DXY,Real_Yield_10Y,Breakeven_10Y,Fed_Funds,WTI_Oil,SP500,VIX,CPI_YoY,M2_YoY
2007-02-28,669.400024,97.2386,2.20,2.36,5.41,61.78,1406.819946,15.420000,2.075765,5.768116
2007-03-01,662.299988,97.3358,2.19,2.37,5.31,61.97,1403.170044,15.820000,2.075765,5.768116
2007-03-02,641.500000,97.3691,2.15,2.37,5.23,61.58,1387.170044,18.610001,2.075765,5.768116
2007-03-03,641.500000,97.3691,2.15,2.37,5.23,61.58,1387.170044,18.610001,2.075765,5.768116
2007-03-04,641.500000,97.3691,2.15,2.37,5.23,61.58,1387.170044,18.610001,2.075765,5.768116


In [17]:
final_data.isna().sum()

Gold              0
DXY               0
Real_Yield_10Y    0
Breakeven_10Y     0
Fed_Funds         0
WTI_Oil           0
SP500             0
VIX               0
CPI_YoY           0
M2_YoY            0
dtype: int64

In [15]:
gold.tail()

Ticker,GC=F
Date,
2026-02-18,4986.500000
2026-02-19,4975.899902
2026-02-20,5059.299805
2026-02-23,5204.700195
2026-02-24,5155.799805


In [18]:
dxy.tail(10)

2026-02-09    117.6392
2026-02-10    117.5216
2026-02-11    117.4601
2026-02-12    117.5376
2026-02-13    117.5258
2026-02-16         NaN
2026-02-17    117.7375
2026-02-18    117.8426
2026-02-19    118.2354
2026-02-20    117.9917
dtype: float64

In [9]:
dxy.head()

2006-01-02    101.4155
2006-01-03    100.7558
2006-01-04    100.2288
2006-01-05    100.2992
2006-01-06    100.0241
dtype: float64

In [12]:
gold2 = yf.download("GC=F", start=START_DATE, end=END_DATE)["Close"]

[*********************100%***********************]  1 of 1 completed


In [13]:
gold2.head()

Ticker,GC=F
Date,
2006-01-03,530.700012
2006-01-04,533.900024
2006-01-05,526.299988
2006-01-06,539.700012
2006-01-09,549.099976
